In [1]:
import time
import os
import subprocess
import sys
import re
import argparse
import collections
import gzip
import math
import shutil
import matplotlib.pyplot as plt
import wandb
import numpy as np
import time
from datetime import datetime
import random

import seaborn as sns
%matplotlib inline
import logging
from silence_tensorflow import silence_tensorflow
#silence_tensorflow()
#os.environ['TPU_LOAD_LIBRARY']='0'
os.environ['TF_ENABLE_EAGER_CLIENT_STREAMING_ENQUEUE']='False'
import tensorflow as tf


import tensorflow.experimental.numpy as tnp
import tensorflow_addons as tfa
from tensorflow import strings as tfs
from tensorflow.keras import mixed_precision
from scipy.stats.stats import pearsonr  
from scipy.stats.stats import spearmanr  
## custom modules
import src.aformer_dual as aformer
#import src.aformer_TF as aformer
from src.layers.layers_dual import *
import src.metrics as metrics
from src.optimizers import *
import src.schedulers as schedulers
import src.utils as utils

import training_utils_aformer_TF_expression as training_utils

from scipy import stats


2022-10-06 01:32:55.535036: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-06 01:32:55.535099: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/opt/conda/lib/python3.7/site-packages/tensorflow_addons/utils/ensure_tf_install.py:67: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.6.0 and strictly below 2.9.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.9.1 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the

In [2]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='node-1')
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver)

with strategy.scope():
    options = tf.data.Options()
    options.experimental_distribute.auto_shard_policy=\
        tf.data.experimental.AutoShardPolicy.FILE
    options.deterministic=False
    options.experimental_threading.max_intra_op_parallelism=1
    mixed_precision.set_global_policy('mixed_bfloat16')
    #options.num_devices = 64

    BATCH_SIZE_PER_REPLICA = 8 # batch size 24, use LR ~ 2.5 e -04
    NUM_REPLICAS = strategy.num_replicas_in_sync
    GLOBAL_BATCH_SIZE = BATCH_SIZE_PER_REPLICA * NUM_REPLICAS
    

2022-10-06 01:32:57.738252: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-10-06 01:32:57.738368: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-10-06 01:32:57.738391: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (tpu-genformer-v2-6): /proc/driver/nvidia/version does not exist


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


2022-10-06 01:32:58.037360: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-06 01:32:58.058008: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:272] Initialize GrpcChannelCache for job worker -> {0 -> 10.30.154.90:8470}
2022-10-06 01:32:58.058047: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:272] Initialize GrpcChannelCache for job localhost -> {0 -> localhost:44434}
2022-10-06 01:32:58.074070: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:272] Initialize GrpcChannelCache for job worker -> {0 -> 10.30.154.90:8470}
2022-10-06 01:32:58.074107: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:272] Initialize GrpcChannelCache for job localhost -> {0 -> localhost:44434}
2022-10-

INFO:tensorflow:Initializing the TPU system: node-1


INFO:tensorflow:Initializing the TPU system: node-1


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [3]:
with strategy.scope():
    data_dict_tr,data_dict_val = training_utils.return_distributed_iterators("gs://picard-testing-176520/seqtoatac_98k_73kstride_blacklist0.25/preprocessed",
                                                                             "gs://picard-testing-176520/seqtoatac_98k_73kstride_blacklist0.25/preprocessed",
                                                                             "atac_only",
                                                                              GLOBAL_BATCH_SIZE,
                                                                              98304,
                                                                              768,
                                                                              128,
                                                                             20,
                                                                               8,
                                                                               1,
                                                                               strategy,
                                                                               options)


In [4]:
with strategy.scope():
    inits = training_utils.get_initializers("/home/jupyter/dev/BE_CD69_paper_2022/enformer_fine_tuning/checkpoint/sonnet_weights")
    model = aformer.aformer(kernel_transformation = 'softmax_kernel_transformation',
                            tf_module_kernel = 'softmax_kernel_transformation',
                            dropout_rate = 0.35,
                            attention_dropout_rate = 0.25,
                            input_length = 98304,
                            atac_length_uncropped = 768,
                            atac_output_length = 448,
                             num_heads = 8,
                             numerical_stabilizer = 0.001,
                             nb_random_features = 256,
                             hidden_size = 384,
                             transformer_depth_1 = 4,
                             transformer_depth_2 = 4,
                             shared_transformer_depth = 4,
                             pre_transf_channels = 384,
                             d_model = 384,
                             TF_inputs=96,
                             norm=True,
                             dim = 48, 
                             max_seq_length = 768,
                             rel_pos_bins=768, 
                             use_rot_emb = True,
                             use_mask_pos = False, 
                             normalize = True,
                             seed = 3,
                             load_init=False,
                             inits=None,
                             filter_list=[192,224,256,288,320,384],
                             freeze_conv_layers=False)

ListWrapper([192, 224, 256, 288, 320, 384])


In [5]:
with strategy.scope():
    scheduler= tf.keras.optimizers.schedules.CosineDecay(
        initial_learning_rate=5.0e-05,
        decay_steps=100000, alpha=1.0)
    scheduler=WarmUp(initial_learning_rate=5.0e-05,
                                 warmup_steps=50,
                                 decay_schedule_fn=scheduler)


    optimizer = tf.keras.optimizers.Adam(learning_rate=scheduler)

    

In [6]:
with strategy.scope():
    metric_dict = {}
    dist_train_step_atac, dist_val_step_atac,\
            dist_train_step_rna,dist_val_step_rna,\
                dist_train_step_both,dist_val_step_both, metric_dict = training_utils.return_train_val_functions(model,
                                                                                               optimizer,
                                                                                               strategy,
                                                                                               metric_dict,
                                                                                               300, 
                                                                                               150,
                                                                                               50,
                                                                                               GLOBAL_BATCH_SIZE,
                                                                                               5.0,
                                                                                               448,
                                                                                               120,
                                                                                               BATCH_SIZE_PER_REPLICA,
                                                                                               rna_loss_scale=None)


In [ ]:
from scipy.stats.stats import pearsonr  
with strategy.scope():
    def sum_log(x):
        return np.log10(1.0 + np.nansum(x))
    
    global_step = 0
    val_losses = []
    val_pearsons = []
    val_R2 = []
    patience_counter = 0
    stop_criteria = False
    best_epoch = 0

    for epoch_i in range(1, 15):
        start = time.time()
        #if epoch_i > 2 : 
        dist_train_step_atac(data_dict_tr)

        print('hg_train_loss: ' + str(metric_dict['hg_tr'].result().numpy()))

        dist_val_step_atac(data_dict_val)

        print('hg_val_loss: ' + str(metric_dict['hg_val'].result().numpy()))
        print('hg_ATAC_pearsons: ' + str(metric_dict['hg_pearsonsR_ATAC'].result()['PearsonR'].numpy()))
        print('hg_ATAC_R2: ' + str(metric_dict['hg_R2_ATAC'].result()['R2'].numpy()))
        
        end = time.time()
        duration = (end - start) / 60.
        print('completed epoch ' + str(epoch_i))
        print('duration(mins): ' + str(duration))
        print('patience counter at: ' + str(patience_counter))

        for key, item in metric_dict.items():
            item.reset_state()

hg_train_loss: 0.08034078
hg_val_loss: 0.08066964
hg_ATAC_pearsons: [0.25985697]
hg_ATAC_R2: [0.05498761]
completed epoch 1
duration(mins): 7.163986921310425
patience counter at: 0
hg_train_loss: 0.050059896
hg_val_loss: 0.080918714
hg_ATAC_pearsons: [0.24193926]
hg_ATAC_R2: [0.05739522]
completed epoch 2
duration(mins): 6.103077205022176
patience counter at: 0
hg_train_loss: 0.04452472
hg_val_loss: 0.041107908
hg_ATAC_pearsons: [0.3555071]
hg_ATAC_R2: [0.12555128]
completed epoch 3
duration(mins): 6.088467593987783
patience counter at: 0
